# Лабораторная Работа №10

## Параметры и импорты

In [9]:
# === ПАРАМЕТРЫ (можно менять под свои условия) ===============================
from pathlib import Path

# Путь к исходному датасету. Можно указать .xlsx или .csv
DATA_PATH = Path('recipes_full.csv')

# Директория, куда будут сохраняться части
OUT_DIR = Path('results/')

# Сколько частей делаем (примерно одинаковых по объёму)
N_PARTS = 8

# Разделитель в выходных CSV (по заданию — точка с запятой)
DELIM = ';'

# Имя шаблона выходных файлов
PART_PREFIX = 'id_tag_nsteps_'
# ============================================================================

In [10]:
# Импорты. openpyxl нужен только для .xlsx
import csv, ast, time, os
from typing import Dict, Tuple, Iterable, List
from multiprocessing import Process, Queue, cpu_count

try:
    from openpyxl import load_workbook  # Для потокового чтения XLSX
except Exception:
    load_workbook = None  # Если пакет не установлен, остаётся поддержка только CSV

# Создаём выходную директорию (если её ещё нет)
OUT_DIR.mkdir(parents=True, exist_ok=True)

## Универсальный потоковый итератор для `.xlsx` и `.csv`

In [11]:
def iter_recipes_unified(path: Path):
    """Итератор, который **построчно** (стримингово) возвращает словари вида:
        {'id': int, 'n_steps': int, 'tags': list[str]}

    Поддерживает два формата источника:
      • XLSX: каждая ячейка первой колонки содержит целую CSV-строку набора рецептов;
               мы читаем лист через openpyxl в режиме read_only и парсим каждую строку как CSV.
      • CSV : обычный CSV-файл с заголовком; читаем заголовок, находим индексы нужных колонок
               и далее читаем строки без загрузки всего файла в память.

    Такой подход не требует больших ресурсов и позволяет обрабатывать очень крупные файлы.
    """
    suf = path.suffix.lower()

    # --------------------------------- XLSX ----------------------------------
    if suf in ('.xlsx', '.xls'):
        if load_workbook is None:
            raise RuntimeError('openpyxl недоступен: нельзя прочитать XLSX')

        # Открываем книгу в режиме "только чтение" (эффективно по памяти)
        wb = load_workbook(filename=path, read_only=True, data_only=True)
        ws = wb.active  # используем активный лист

        first = True  # флаг для пропуска заголовка (первая CSV-строка в файле)
        try:
            # Итерация по строкам листа отдаёт кортежи со значениями ячеек
            for row in ws.iter_rows(values_only=True):
                if not row:
                    continue  # пустая строка
                line = row[0]  # в наборе — всё содержится в первой колонке
                if line is None:
                    continue

                # В ячейке лежит полноценная CSV-строка — парсим её корректно (учёт кавычек).
                for cols in csv.reader([line]):
                    if first:
                        first = False  # это заголовок; пропускаем и двигаемся дальше
                        continue
                    # Индексы полей соответствуют структуре набора:
                    # name(0), id(1), minutes(2), contributor_id(3), submitted(4),
                    # tags(5), n_steps(6), steps(7), description(8), ingredients(9), n_ingredients(10)
                    try:
                        rid = int(cols[1])         # id
                        n_steps = int(cols[6])     # количество шагов
                    except Exception:
                        # Если парсинг не удался — пропускаем строку
                        continue
                    # Разбираем список тегов из строкового представления Python-списка
                    try:
                        tags = ast.literal_eval(cols[5])
                        if not isinstance(tags, list):
                            tags = []
                    except Exception:
                        tags = []

                    # Возвращаем унифицированный словарь
                    yield {'id': rid, 'n_steps': n_steps, 'tags': tags}
        finally:
            wb.close()  # обязательно закрываем книгу

    # ---------------------------------- CSV ----------------------------------
    elif suf == '.csv':
        # Для CSV используем csv.reader (не DictReader), чтобы точно работать с заголовком
        # и позициями колонок — это надёжнее, если в данных встречаются необычные кавычки/разделители.
        def open_csv(p: Path):
            # Пытаемся учесть BOM (utf-8-sig); если его нет — читаем обычным utf-8
            try:
                return open(p, 'r', encoding='utf-8-sig', newline='')
            except Exception:
                return open(p, 'r', encoding='utf-8', newline='')

        with open_csv(path) as f:
            reader = csv.reader(f)
            header = next(reader, None)  # читаем строку заголовка
            if not header:
                return  # пустой файл

            # Построим карту "имя колонки -> индекс", нормализовав регистр
            idx = {name.lower(): i for i, name in enumerate(header)}
            i_id    = idx.get('id')
            i_steps = idx.get('n_steps')
            i_tags  = idx.get('tags')
            if i_id is None or i_steps is None or i_tags is None:
                raise ValueError('В CSV нет необходимых колонок: id, n_steps, tags')

            for cols in reader:
                # Пропускаем строки, где не хватает колонок
                if len(cols) <= max(i_id, i_steps, i_tags):
                    continue
                # Аккуратно приводим типы; иногда n_steps может прийти как '11.0'
                try:
                    rid = int(cols[i_id])
                    n_steps = int(float(cols[i_steps]))
                except Exception:
                    continue
                # Разбираем список тегов (формат: "['tag1','tag2', ...]")
                try:
                    tags = ast.literal_eval(cols[i_tags])
                    if not isinstance(tags, list):
                        tags = []
                except Exception:
                    tags = []

                yield {'id': rid, 'n_steps': n_steps, 'tags': tags}

    else:
        # Если передали что-то отличное от .xlsx/.csv — сообщаем пользователю
        raise ValueError(f'Неподдерживаемое расширение: {suf}')

## Задание 1 — разбиение на части `id_tag_nsteps_*.csv`
Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```

In [12]:
def split_to_parts_id_tag_nsteps_unified(src_path: Path, out_dir: Path, n_parts: int = 8) -> List[Path]:
    """Читает источники (.xlsx или .csv) **потоково** и создаёт `n_parts` файлов
    с колонками `id;tag;n_steps`. Заполнение частей идёт по схеме **round-robin** —
    по очереди в каждый файл, чтобы они получались примерно одинакового размера,
    даже если заранее неизвестно общее число строк.
    """
    # Готовим выходные файлы и csv.writer'ы
    out_paths = [out_dir / f"{PART_PREFIX}{i+1}.csv" for i in range(n_parts)]
    writers, files = [], []
    try:
        for p in out_paths:
            f = open(p, 'w', newline='', encoding='utf-8')
            files.append(f)
            w = csv.writer(f, delimiter=DELIM)
            w.writerow(['id','tag','n_steps'])  # пишем заголовок
            writers.append(w)

        # Индекс текущего файла для round-robin (0..n_parts-1)
        rr = 0

        # Идём по рецептам и раскладываем пары (id, tag, n_steps)
        for rec in iter_recipes_unified(src_path):
            rid = rec['id']
            n_steps = rec['n_steps']
            tags = rec['tags'] or []
            if rid is None or n_steps is None:
                continue  # перестраховка: пропустить странную строку

            for tag in tags:
                if not tag:
                    continue
                t = str(tag).strip()
                if not t:
                    continue
                # Записываем строку и переходим к следующему «ведру»
                writers[rr].writerow([rid, t, n_steps])
                rr = (rr + 1) % n_parts
    finally:
        # Гарантированно закрываем все файлы (даже при исключениях)
        for f in files:
            try:
                f.close()
            except Exception:
                pass

    return out_paths

# Выполним разбиение прямо сейчас
parts = split_to_parts_id_tag_nsteps_unified(DATA_PATH, OUT_DIR, N_PARTS)
print('Созданы части:', [p.name for p in parts])
print('Размеры (байт):', [p.stat().st_size for p in parts])

Созданы части: ['id_tag_nsteps_1.csv', 'id_tag_nsteps_2.csv', 'id_tag_nsteps_3.csv', 'id_tag_nsteps_4.csv', 'id_tag_nsteps_5.csv', 'id_tag_nsteps_6.csv', 'id_tag_nsteps_7.csv', 'id_tag_nsteps_8.csv']
Размеры (байт): [38846621, 38860091, 38844895, 38853250, 38849311, 38854677, 38848220, 38845459]


## Задание 2 — среднее `n_steps` по тегам для **одного** файла
Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [13]:
from typing import Dict, Tuple

def tag_stats_from_part(csv_path: Path) -> Dict[str, Tuple[int,int]]:
    """Считает промежуточные статистики по одному файлу-части:
       возвращает словарь `tag -> (sum_steps, count)`.
       Такой формат удобен для последующего корректного объединения результатов.
    """
    stats: Dict[str, Tuple[int,int]] = {}
    with open(csv_path, 'r', encoding='utf-8', newline='') as f:
        r = csv.reader(f, delimiter=DELIM)
        next(r, None)  # пропускаем заголовок
        for row in r:
            if len(row) != 3:
                continue
            _rid, tag, n_steps = row
            if not tag:
                continue
            try:
                n = int(n_steps)
            except Exception:
                continue
            # Накапливаем сумму шагов и количество вхождений для тега
            s,c = stats.get(tag, (0,0))
            stats[tag] = (s+n, c+1)
    return stats

def tag_means_from_part(csv_path: Path) -> Dict[str, float]:
    """Возвращает `tag -> среднее n_steps` для одного файла."""
    stats = tag_stats_from_part(csv_path)
    return {t: (s / c) for t, (s, c) in stats.items() if c > 0}

# Мини-проверка на первой части
example = tag_means_from_part(parts[0])
print('Пример 10 тегов из', parts[0].name, ':')
for k in list(example.keys())[:10]:
    print(k, f"-> {example[k]:.2f}")

Пример 10 тегов из id_tag_nsteps_1.csv :
mexican -> 5.32
ham-and-bean-soup -> 3.53
quick-breads -> 5.02
60-minutes-or-less -> 9.50
dinner-party -> 8.22
course -> 9.25
chicken -> 7.26
veal -> 3.61
dips-summer -> 3.56
bacon -> 4.00


## Задание 3 — объединение результатов по всем файлам (последовательно)
Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы иметь возможность получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.

In [14]:
def merge_tag_stats(dicts: Iterable[Dict[str, Tuple[int,int]]]) -> Dict[str, Tuple[int,int]]:
    """Складывает несколько `tag -> (sum, count)` в один словарь."""
    merged: Dict[str, Tuple[int,int]] = {}
    for d in dicts:
        for tag, (s, c) in d.items():
            S, C = merged.get(tag, (0, 0))
            merged[tag] = (S + s, C + c)
    return merged

def sequential_all_means(files: List[Path]):
    """Последовательно обрабатывает все части и считает финальные средние по тегам."""
    t0 = time.perf_counter()
    parts_stats = [tag_stats_from_part(p) for p in files]
    merged = merge_tag_stats(parts_stats)
    means = {t: (s / c) for t, (s, c) in merged.items() if c > 0}
    dt = time.perf_counter() - t0
    return means, dt

means_seq, t_seq = sequential_all_means(parts)
print(f'Последовательно: {len(parts)} файлов за {t_seq:.4f} с; тегов: {len(means_seq)}')
print('Примеры 10 тегов:')
for k in list(means_seq.keys())[:10]:
    print(k, f"-> {means_seq[k]:.2f}")

Последовательно: 8 файлов за 10.6719 с; тегов: 551
Примеры 10 тегов:
mexican -> 5.30
ham-and-bean-soup -> 3.51
quick-breads -> 5.06
60-minutes-or-less -> 9.41
dinner-party -> 8.23
course -> 9.27
chicken -> 7.33
veal -> 3.68
dips-summer -> 3.48
bacon -> 4.11


## Задание 4 — `multiprocessing`: отдельный процесс на каждый файл
Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [15]:
from multiprocessing import Process, Queue, cpu_count

def worker_file_stats(path: str, out_q: Queue):
    """Рабочий процесс: считает `tag_stats_from_part` и складывает результат в очередь."""
    out_q.put(tag_stats_from_part(Path(path)))

def mp_one_proc_per_file(files: List[Path]):
    """Запускает по **отдельному процессу на каждый файл**.
       Коммуникация через очередь: каждый процесс кладёт в неё словарь статистик.
    """
    t0 = time.perf_counter()
    q = Queue()
    procs = []
    for p in files:
        pr = Process(target=worker_file_stats, args=(str(p), q))
        pr.start()
        procs.append(pr)

    # Забираем результаты от всех процессов
    collected = [q.get() for _ in files]

    # Дожидаемся завершения
    for pr in procs:
        pr.join()

    merged = merge_tag_stats(collected)
    means = {t: (s / c) for t, (s, c) in merged.items() if c > 0}
    dt = time.perf_counter() - t0
    return means, dt

means_mp1, t_mp1 = mp_one_proc_per_file(parts)
print(f'mp(one-per-file): {len(parts)} файлов за {t_mp1:.4f} с; тегов: {len(means_mp1)}')

mp(one-per-file): 8 файлов за 11.5837 с; тегов: 551


## Задание 5 — `multiprocessing` с фиксированным числом процессов и очередями
(*) Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы.

In [16]:
def worker_fixed(in_q: Queue, out_q: Queue):
    """Рабочий процесс: берёт путь к файлу из входной очереди, считает статистику
       и кладёт результат в выходную очередь. Завершается при получении `None`.
    """
    while True:
        path = in_q.get()
        if path is None:
            break  # «ядовитая пилюля» — сигнал завершиться
        out_q.put(tag_stats_from_part(Path(path)))

def mp_fixed_workers(files: List[Path], n_workers: int | None = None):
    """Запускает фиксированное число рабочих процессов (по умолчанию: половина ядер).
       Задания подаются через очередь `in_q`, результаты собираются из `out_q`.
    """
    if n_workers is None:
        n_workers = max(1, cpu_count() // 2)

    t0 = time.perf_counter()
    in_q, out_q = Queue(), Queue()

    # Стартуем пул рабочих процессов
    procs = [Process(target=worker_fixed, args=(in_q, out_q)) for _ in range(n_workers)]
    for pr in procs:
        pr.start()

    # Подаём задания (пути) в очередь
    for p in files:
        in_q.put(str(p))

    # Отправляем каждому процессу «ядовитую пилюлю» — указание завершиться
    for _ in procs:
        in_q.put(None)

    # Собираем результаты
    collected = [out_q.get() for _ in files]

    # Дожидаемся завершения всех процессов
    for pr in procs:
        pr.join()

    merged = merge_tag_stats(collected)
    means = {t: (s / c) for t, (s, c) in merged.items() if c > 0}
    dt = time.perf_counter() - t0
    return means, dt

means_mp2, t_mp2 = mp_fixed_workers(parts)
print(f'mp(fixed-workers): {len(parts)} файлов за {t_mp2:.4f} с; тегов: {len(means_mp2)}')
print('Совпадение ключей:', set(means_seq.keys()) == set(means_mp1.keys()) == set(means_mp2.keys()))

mp(fixed-workers): 8 файлов за 9.8272 с; тегов: 551
Совпадение ключей: True
